In [16]:
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, make_scorer
import pandas as pd
import numpy as np
import joblib

# 1.AD vs NC

In [17]:
data_dir = '/Users/imdohyeon/Documents/PythonWorkspace/4n/preprocessSeg/extracted/ver3(binary without CSP features)/features_binary_ad_nc.csv'
data = pd.read_csv(data_dir)

In [18]:
data

,kurtosis_Fp1,kurtosis_Fp2,kurtosis_F7,kurtosis_F3,kurtosis_Fz,kurtosis_F4,kurtosis_F8,kurtosis_T7,kurtosis_C3,kurtosis_Cz,...,spectral_entropy_C4,spectral_entropy_T8,spectral_entropy_P7,spectral_entropy_P3,spectral_entropy_Pz,spectral_entropy_P4,spectral_entropy_P8,spectral_entropy_O1,spectral_entropy_O2,label
0,-0.418358,0.332468,-0.342700,-0.434762,-0.363083,1.541473,3.803859,-0.366574,-0.361034,-0.317023,...,2.769378,3.243881,3.314180,2.991150,2.867881,2.897658,3.226472,3.215344,3.264944,1
1,-0.553936,0.645437,-0.499525,-0.413223,-0.233330,1.566611,3.691952,-0.446604,-0.195897,-0.095939,...,3.014827,3.851837,3.643615,3.271922,3.141803,3.162673,3.711791,3.533822,3.642025,1
2,1.028320,-0.381386,0.390925,0.612009,0.007393,-1.033033,-0.263930,-0.269617,-0.003700,-1.806611,...,3.997216,3.996011,4.050526,4.029060,4.035325,4.044881,4.029477,4.004199,4.097106,1
3,1.024712,-0.350840,0.469474,0.555023,-0.059696,-0.864880,0.112843,-0.260772,-0.060767,-1.878602,...,3.781055,3.795044,3.863689,3.835626,3.824710,3.850665,3.836143,3.776048,3.924873,1
4,-0.007232,0.071214,-0.244308,-0.243306,-0.211503,-0.158064,4.221788,-0.248971,-0.280709,-0.297559,...,2.857713,3.287834,3.464297,3.088678,2.997639,3.016220,3.331265,3.395650,3.406698,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4166,-0.400714,-0.344705,0.841186,3.154313,-0.749571,-0.234422,-0.933491,-0.815349,-0.923803,-0.869718,...,4.432919,5.570556,4.419069,4.430614,4.181929,4.305306,4.995574,4.345621,4.280138,0
4167,0.196473,0.205968,-0.230757,0.297017,-0.288704,-1.187940,-1.216336,0.340628,-1.160514,-1.294505,...,4.434355,5.380242,4.572584,4.443683,4.153778,4.289267,4.884699,4.500191,4.388379,0
4168,-0.159535,-0.032668,0.049289,0.301747,-1.062893,-0.308668,-1.548534,-0.540624,-0.500203,-1.501149,...,4.299190,5.376271,4.427081,4.419950,4.050663,4.196556,4.726267,4.345120,4.224140,0
4169,-0.689377,-0.455913,-0.331188,-0.298873,-1.089699,-0.470368,-0.442167,-0.021996,-0.644215,-0.376904,...,5.149351,5.042029,4.876800,4.801681,5.172031,4.664239,4.391314,4.659940,4.639465,0


- Multi class에서 L1 정규화 적용 시 모든 주파수 도메인 피처를 제거하였음. 주파수 도메인 피처보다 다른 피처가 더 중요하다고 판단되어, 주파수 도메인 피처는 추출하지 않고 진행. 

In [19]:
# 데이터 셔플
data = data.sample(frac=1).reset_index(drop=True)
data

,kurtosis_Fp1,kurtosis_Fp2,kurtosis_F7,kurtosis_F3,kurtosis_Fz,kurtosis_F4,kurtosis_F8,kurtosis_T7,kurtosis_C3,kurtosis_Cz,...,spectral_entropy_C4,spectral_entropy_T8,spectral_entropy_P7,spectral_entropy_P3,spectral_entropy_Pz,spectral_entropy_P4,spectral_entropy_P8,spectral_entropy_O1,spectral_entropy_O2,label
0,-0.527302,1.181890,-0.177920,-1.510379,-1.721818,0.183468,2.225843,-0.098798,-0.460286,1.256587,...,3.376487,3.640987,3.888772,3.723262,3.659794,3.691210,3.767603,3.907507,3.948341,0
1,-0.468375,-0.432942,4.190411,-0.413157,-0.419468,-0.377227,-0.167427,-0.062911,-0.231348,-0.359514,...,4.954618,5.253048,4.804027,5.083363,4.788634,4.848915,5.223115,4.668203,4.702218,0
2,1.155203,2.960920,-0.654380,-0.680629,0.715378,-0.100891,-0.459587,0.540402,-1.126710,0.803980,...,5.221303,5.342423,5.263270,4.931122,4.734398,4.924445,4.999133,5.202016,4.960051,0
3,-0.458505,-0.057117,-0.571557,0.897342,-0.380009,-0.678935,-0.295140,0.287175,-1.357305,-0.316765,...,5.183495,4.835472,4.939544,4.640270,4.201044,4.655578,4.950434,4.831939,4.866074,0
4,0.885077,-0.758681,1.312403,0.888432,0.160801,-1.528173,-2.169484,1.452818,1.207978,0.920795,...,3.282496,3.301596,3.414393,3.416504,3.434186,3.337961,3.346638,3.472277,3.465620,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4166,-0.327192,2.271211,-0.791755,-0.482616,3.058097,0.497249,0.265233,0.559257,-0.734394,-0.403077,...,4.804701,4.785434,5.619818,5.604437,5.005377,5.073753,5.179712,5.800539,5.194397,0
4167,0.449526,-1.075555,-0.109794,1.304127,0.394536,-0.816568,-1.794473,0.904070,1.968371,-0.734859,...,2.887299,3.010544,3.023207,2.934649,2.916986,2.941613,3.012629,3.001328,3.014647,1
4168,0.834975,0.469620,0.346890,0.857451,1.459243,1.333481,1.980542,-0.162555,-0.191750,-0.376238,...,2.930830,2.971452,2.981269,2.943016,2.939817,2.947428,2.966581,2.951815,2.960929,1
4169,-0.577184,-2.773130,-2.097057,-0.784321,-0.416779,-0.580253,-0.339057,0.729322,0.449912,0.670737,...,2.629997,2.662707,2.869745,2.780360,2.716437,2.719608,2.837668,2.960365,2.802560,0


In [20]:
# 데이터 나누기
X = data.drop(['label'], axis=1)
y = data['label']
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### 1-1. L1 Regularization + RF (Grid)

In [21]:
# Lasso를 사용한 특징 선택
lasso = Lasso(max_iter=10000)

# Random Forest 모델
rf_model = RandomForestClassifier(random_state=42)

# 파이프라인 구축
pipeline = Pipeline([
    ('feature_selection', SelectFromModel(estimator=lasso)),
    ('random_forest', rf_model)
])

# 하이퍼파라미터 그리드 정의
param_grid = {
    'feature_selection__estimator__alpha': [0.001, 0.01, 0.1, 1],  # Lasso의 alpha 값
    'random_forest__n_estimators': [50, 100, 200],  # Random Forest의 트리 개수
    'random_forest__max_depth': [None, 10, 20, 30]  # Random Forest의 최대 깊이
}
# Weighted F1 스코어 사용
f1_scorer = make_scorer(f1_score, average='weighted')

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring=f1_scorer, n_jobs=-1, verbose=1)

In [22]:
# 모델 학습
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


/Users/imdohyeon/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
60 fits failed out of a total of 240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
48 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/imdohyeon/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/imdohyeon/miniconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/imdohyeon/miniconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py", line 328, in fit
    X, y

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('feature_selection',
                                        SelectFromModel(estimator=Lasso(max_iter=10000))),
                                       ('random_forest',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'feature_selection__estimator__alpha': [0.001, 0.01,
                                                                 0.1, 1],
                         'random_forest__max_depth': [None, 10, 20, 30],
                         'random_forest__n_estimators': [50, 100, 200]},
             scoring=make_scorer(f1_score, average=weighted), verbose=1)

In [23]:
# 최적 파라미터 및 성능 출력
print("Best Parameters:", grid_search.best_params_)
print("Best CV Score:", grid_search.best_score_)

# 최적 모델
best_model_rf = grid_search.best_estimator_

# 테스트 세트 평가
y_pred = grid_search.best_estimator_.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Test Accuracy: {accuracy:.2f}", f"Test F1: {f1:.2f}")

Best Parameters: {'feature_selection__estimator__alpha': 0.001, 'random_forest__max_depth': None, 'random_forest__n_estimators': 200}
Best CV Score: 0.8891188525508511
Test Accuracy: 0.89 Test F1: 0.89


In [24]:
joblib.dump(best_model_rf, "/Users/imdohyeon/Documents/PythonWorkspace/4n/model/best_l1rf_model_ver3.pkl")
print("Model saved to 'best_l1rf_model_ver3.pkl'")

Model saved to 'best_l1rf_model_ver3.pkl'


## 1-2. L1 Regularization + XGB(Grid)

In [25]:
import xgboost as xgb
from xgboost import XGBClassifier

In [26]:
# Lasso를 사용한 특징 선택
lasso = Lasso(max_iter=10000)

# XGBoost 모델
xgb_model = XGBClassifier(
    objective='binary:logistic',
    use_label_encoder=False,
    eval_metric='logloss'
)

# 파이프라인
pipeline = Pipeline([
    ('feature_selection', SelectFromModel(estimator=lasso)),
    ('xgboost', xgb_model)
])

# Grid Search 설정
param_grid = {
    'feature_selection__estimator__alpha': [0.001, 0.01, 0.1],
    'xgboost__n_estimators': [50, 100, 200],
    'xgboost__max_depth': [3, 5, 7],
    'xgboost__learning_rate': [0.01, 0.1, 0.3]
}

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'f1_score': make_scorer(f1_score, average='weighted')
}

grid_search_xgb = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring=scoring,
    refit='accuracy',
    verbose=1,
    n_jobs=-1
)

In [27]:
# 모델 학습
grid_search_xgb.fit(X_train, y_train)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('feature_selection',
                                        SelectFromModel(estimator=Lasso(max_iter=10000))),
                                       ('xgboost',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      callbacks=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      early_stopping_rounds=None,
                                                      enable_categorical=False,
                                                      eval_metric='logloss',
                                                      gamma=None, gpu_id=None,
                   

In [28]:
# 최적 파라미터 출력
print("Best Parameters:", grid_search_xgb.best_params_)

# 테스트 데이터 평가
y_pred = grid_search_xgb.best_estimator_.predict(X_test)
print(f"Test Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print(f"Test F1-Score: {f1_score(y_test, y_pred, average='weighted'):.2f}")

Best Parameters: {'feature_selection__estimator__alpha': 0.001, 'xgboost__learning_rate': 0.1, 'xgboost__max_depth': 7, 'xgboost__n_estimators': 200}
Test Accuracy: 0.91
Test F1-Score: 0.91


In [29]:
# 최적 모델 저장
joblib.dump(grid_search_xgb.best_estimator_, "/Users/imdohyeon/Documents/PythonWorkspace/4n/model/best_l1xgb_model_ver3.pkl")
print("Model saved to 'best_l1xgb_model_ver3.pkl'")

Model saved to 'best_l1xgb_model_ver3.pkl'


# 2. MCI vs NC

## 2-1. L1 + RF(Grid)

In [31]:
data_mci_dir = '/Users/imdohyeon/Documents/PythonWorkspace/4n/preprocessSeg/extracted/ver3(binary without CSP features)/features_binary_mci_nc.csv'
data_mci = pd.read_csv(data_mci_dir)

In [32]:
data_mci = data_mci.sample(frac=1).reset_index(drop=True)
# 데이터 나누기
X_mci = data_mci.drop(['label'], axis=1)
y_mci = data_mci['label']
# 데이터 분할
X_train_mci, X_test_mci, y_train_mci, y_test_mci = train_test_split(X_mci, y_mci, test_size=0.3, random_state=42)

In [33]:
# Lasso를 사용한 특징 선택
lasso = Lasso(max_iter=10000)

# Random Forest 모델
rf_model = RandomForestClassifier(random_state=42)

# 파이프라인 구축
pipeline = Pipeline([
    ('feature_selection', SelectFromModel(estimator=lasso)),
    ('random_forest', rf_model)
])

# 하이퍼파라미터 그리드 정의
param_grid = {
    'feature_selection__estimator__alpha': [0.001, 0.01, 0.1, 1],  # Lasso의 alpha 값
    'random_forest__n_estimators': [50, 100, 200],  # Random Forest의 트리 개수
    'random_forest__max_depth': [None, 10, 20, 30]  # Random Forest의 최대 깊이
}
# Weighted F1 스코어 사용
f1_scorer = make_scorer(f1_score, average='weighted')

grid_search_rf_mci = GridSearchCV(pipeline, param_grid, cv=5, scoring=f1_scorer, n_jobs=-1, verbose=1)

In [34]:
# 모델 학습
grid_search_rf_mci.fit(X_train_mci, y_train_mci)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


/Users/imdohyeon/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
60 fits failed out of a total of 240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
24 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/imdohyeon/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/imdohyeon/miniconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/imdohyeon/miniconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py", line 328, in fit
    X, y

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('feature_selection',
                                        SelectFromModel(estimator=Lasso(max_iter=10000))),
                                       ('random_forest',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'feature_selection__estimator__alpha': [0.001, 0.01,
                                                                 0.1, 1],
                         'random_forest__max_depth': [None, 10, 20, 30],
                         'random_forest__n_estimators': [50, 100, 200]},
             scoring=make_scorer(f1_score, average=weighted), verbose=1)

In [35]:
# 최적 파라미터 및 성능 출력
print("Best Parameters:", grid_search_rf_mci.best_params_)
print("Best CV Score:", grid_search_rf_mci.best_score_)

# 최적 모델 저장
best_model_rf_mci = grid_search_rf_mci.best_estimator_
joblib.dump(best_model_rf_mci, "/Users/imdohyeon/Documents/PythonWorkspace/4n/model/best_l1rf_model_ver3_mci.pkl")
print("Model saved to 'best_l1rf_model_ver3_mci.pkl'")

# 테스트 세트 평가
y_pred_mci = grid_search_rf_mci.best_estimator_.predict(X_test_mci)
accuracy = accuracy_score(y_test_mci, y_pred_mci)
f1 = f1_score(y_test_mci, y_pred_mci, average='weighted')
print(f"Test Accuracy: {accuracy:.2f}", f"Test F1: {f1:.2f}")

Best Parameters: {'feature_selection__estimator__alpha': 0.001, 'random_forest__max_depth': 20, 'random_forest__n_estimators': 200}
Best CV Score: 0.9348473670161761
Model saved to 'best_l1rf_model_ver3_mci.pkl'
Test Accuracy: 0.94 Test F1: 0.94


## 2-2. L1 + XGB(Grid)

In [43]:
# Lasso 모델 설정
lasso = Lasso(alpha=0.01, max_iter=10000)

# SelectFromModel 학습
feature_selector = SelectFromModel(estimator=lasso)
feature_selector.fit(X_train_mci, y_train_mci)  # Lasso 모델 학습

SelectFromModel(estimator=Lasso(alpha=0.01, max_iter=10000))

In [44]:
# XGBoost 모델
xgb_model = XGBClassifier(
    objective='binary:logistic',
    use_label_encoder=False,
    eval_metric='logloss'
)

# 파이프라인
pipeline = Pipeline([
    ('feature_selection', SelectFromModel(estimator=lasso)),
    ('xgboost', xgb_model)
])

# Grid Search 설정
param_grid = {
    'feature_selection__estimator__alpha': [0.001, 0.01, 0.1],
    'xgboost__n_estimators': [50, 100, 200],
    'xgboost__max_depth': [3, 5, 7],
    'xgboost__learning_rate': [0.01, 0.1, 0.3]
}

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'f1_score': make_scorer(f1_score, average='weighted')
}

grid_search_xgb_mci = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring=scoring,
    refit='accuracy',
    verbose=1,
    n_jobs=-1
)

In [45]:
# 모델 학습
grid_search_xgb_mci.fit(X_train_mci, y_train_mci)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('feature_selection',
                                        SelectFromModel(estimator=Lasso(alpha=0.01,
                                                                        max_iter=10000))),
                                       ('xgboost',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      callbacks=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      early_stopping_rounds=None,
                                                      enable_categorical=False,
                                                      eval_metric='logloss',
              

In [46]:
# 최적 파라미터 출력
print("Best Parameters:", grid_search_xgb_mci.best_params_)

# 테스트 데이터 평가
y_pred_mci = grid_search_xgb_mci.best_estimator_.predict(X_test_mci)
print(f"Test Accuracy: {accuracy_score(y_test_mci, y_pred_mci):.2f}")
print(f"Test F1-Score: {f1_score(y_test_mci, y_pred_mci, average='weighted'):.2f}")

Best Parameters: {'feature_selection__estimator__alpha': 0.001, 'xgboost__learning_rate': 0.3, 'xgboost__max_depth': 5, 'xgboost__n_estimators': 100}
Test Accuracy: 0.95
Test F1-Score: 0.95


In [47]:
# 최적 모델 저장
joblib.dump(grid_search_xgb_mci.best_estimator_, "/Users/imdohyeon/Documents/PythonWorkspace/4n/model/best_l1xgb_model_ver3_mci.pkl")
print("Model saved to 'best_l1xgb_logloss_model_ver3_mci.pkl'")

Model saved to 'best_l1xgb_logloss_model_ver3_mci.pkl'


# 3. 나머지 절반 NC 데이터 테스트

In [50]:
new_data_dir = '/Users/imdohyeon/Documents/PythonWorkspace/4n/preprocessSeg/extracted/ver3(binary without CSP features)/features_nc2.csv'

In [51]:
new_data = pd.read_csv(new_data_dir)
new_data_X = new_data.drop(['label'], axis=1)
new_data_y = new_data['label']

In [52]:
# AD vs NC
y_pred_rf = grid_search.best_estimator_.predict(new_data_X)
y_pred_xgb = grid_search_xgb.best_estimator_.predict(new_data_X)

In [53]:
# AD vs NC 결과 확인
nc2_acc_rf = accuracy_score(new_data_y, y_pred_rf)
nc2_f1_rf = f1_score(new_data_y, y_pred_rf, average='weighted')

nc2_acc_xgb = accuracy_score(new_data_y, y_pred_xgb)
nc2_f1_xgb = f1_score(new_data_y, y_pred_xgb, average='weighted')

print(f"rf. Test Accuracy: {nc2_acc_rf:.2f}", f"Test F1: {nc2_f1_rf:.2f}")
print(f"xgb. Test Accuracy: {nc2_acc_xgb:.2f}", f"Test F1: {nc2_f1_xgb:.2f}")

rf. Test Accuracy: 0.85 Test F1: 0.92
xgb. Test Accuracy: 0.82 Test F1: 0.90


In [54]:
# MCI vs NC
y_pred_rf_mci = grid_search_rf_mci.predict(new_data_X)
y_pred_xgb_mci = grid_search_xgb_mci.predict(new_data_X)

In [55]:
# AD vs MCI 결과 확인
nc2_acc_rf_mci = accuracy_score(new_data_y, y_pred_rf_mci)
nc2_f1_rf_mci = f1_score(new_data_y, y_pred_rf_mci, average='weighted')

nc2_acc_xgb_mci = accuracy_score(new_data_y, y_pred_xgb_mci)
nc2_f1_xgb_mci = f1_score(new_data_y, y_pred_xgb_mci, average='weighted')

print(f"rf. Test Accuracy: {nc2_acc_rf_mci:.2f}", f"Test F1: {nc2_f1_rf_mci:.2f}")
print(f"xgb. Test Accuracy: {nc2_acc_xgb_mci :.2f}", f"Test F1: {nc2_f1_xgb_mci:.2f}")

rf. Test Accuracy: 0.70 Test F1: 0.82
xgb. Test Accuracy: 0.76 Test F1: 0.86
